# Базовые приёмы работы с данными в Pandas

Предобработка данных и их подготовка к анализу и подаче в модель.

Под предобработкой понимаются следующие этапы работы с данными:

очистка данных от аномальных значений (выбросов);\
работа с пропущенными значениями;\
удаление признаков, которые не несут полезной информации;\
создание новых признаков;\
преобразование признаков и приведение данных к необходимому для анализа и модели формату.

## Feature Engineering

Gри правильных преобразованиях таблицы можно добиваться лучшего качества прогноза, а также извлекать новую информацию из данных и интерпретировать её для заказчика. 

Такой подход часто называют Feature Engineering, или генерацией признаков (фичей).

## Создание копии таблицы

Чтобы не переопределять переменную melb_data и тем самым не повредить первоначальный DataFrame, создадим копию melb_df с помощью метода copy()

melb_df = melb_data.copy()
melb_df.head()

## Удаление столбцов

Важное место занимает возможность удаления столбцов из таблицы.\
Это может быть полезно, например, когда в данных есть признаки, которые не несут полезной информации
За удаление строк и столбцов в таблице отвечает метод drop().

Основные параметры метода drop()

labels — порядковые номера или имена столбцов, которые подлежат удалению; если их несколько, то передаётся список;\
axis — ось совершения операции, axis=0 — удаляются строки, axis=1 — удаляются столбцы;\
inplace — если параметр выставлен на True, происходит замена изначального DataFrame на новый, при этом метод ничего не возвращает;\ 
если на False — возвращается копия DataFrame, из которой удалены указанные строки (столбцы), при этом первоначальный DataFrame не изменяется; по умолчанию параметр равен False.

Пример

melb_df = melb_df.drop(['index', 'Coordinates'], axis=1)
melb_df.head()
Альтернативный вариант:

melb_df.drop(['index','Coordinates'],axis=1,inplace=True)
melb_df.head()

## Математические операции со столбцами

Столбцы можно складывать, вычитать, умножать, делить между собой, а также возводить в степень.\
С помощью таких операций мы можем создавать новые признаки или производить преобразования над старыми.

Причём все операции со столбцами совершаются поэлементно, очень быстро, а самое главное — без написания циклов.

Такая производительность достигается за счёт того, что все математические операции со столбцами выполняются на языке программирования С,\
что значительно повышает скорость вычислений по сравнению с перебором элементов в цикле. 

Пример:
total_rooms = melb_df['Rooms'] + melb_df['Bedroom'] + melb_df['Bathroom']

## Признаки даты и времени

Формат datetime

Многие выгрузки из систем и баз данных имеют свой служебный формат. Например, формат времени из разных систем может отличаться:

2018-11-09 15:45:21;
11/09/2018 3:45:20 PM;
2018-11-09T15:45:21.2984.
Для всех этих случаев необходимо задавать формат распознавания дат и уметь сравнивать их между собой.\
Для этого был создан единый способ обозначения даты и времени. 

Таким форматом в Pandas является формат datetime, который записывается как YYYY-MM-DD HH: MM: SS,\
то есть составляющие времени указываются в следующем порядке: год, месяц, день, час, минута, секунда.

Для того чтобы преобразовывать столбцы с датами, записанными в распространённых форматах,\
в формат datetime, можно воспользоваться функцией pandas.to_datetime().\
В нашем случае в функции нужно указать параметр dayfirst=True, который будет обозначать,\
что в первоначальном признаке первым идет день. Преобразуем столбец Date в формат datetime, передав его в эту функцию:

Пример:
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
display(melb_df['Date'])

## Выделение атрибутов datetime

Тип данных datetime позволяет с помощью специального аксессора dt выделять составляющие времени из каждого элемента столбца, такие как:

date — дата;\
year, month, day — год, месяц, день;\
time — время;\
hour, minute, second — час, минута, секунда;

dayofweek — номер дня недели, от 0 до 6, где 0 — понедельник, 6 — воскресенье;\
day_name — название дня недели;\
dayofyear — порядковый день года;\
quarter — квартал (интервал в три месяца).

Пример:
years_sold = melb_df['Date'].dt.year
print(years_sold)
print('Min year sold:', years_sold.min())
print('Max year sold:', years_sold.max())
print('Mode year sold:', years_sold.mode()[0])

## Работа с интервалами

Часто бывает такая ситуация, что необходимо вычислять интервалы между двумя временными промежутками. Например, можно вычислить,\
сколько дней прошло с 1 января 2016 года до момента продажи объекта. Для этого можно просто найти разницу между датами продаж и\
заявленной датой, представленной в формате datetime:

Пример:
delta_days = melb_df['Date'] - pd.to_datetime('2016-01-01') 
display(delta_days)

## Создание и преобразование столбцов с помощью функций

Мы можем написать некоторую функцию, которая принимает на вход один элемент столбца, каким-то образом его обрабатывает и возвращает результат, после чего применить эту функцию к каждому элементу в столбце с помощью специального метода **apply()**

Количество уникальных значений в столбце с можно вычислить помощью метода **nunique()**